In [ ]:
%pylab inline

import os
import pandas as pd
import pickle
pj = os.path.join
import json

import seaborn as sns
sns.set(style="ticks",font_scale=1.5,font='Helvetica')
sns.set_palette(sns.hls_palette(8, l=.3, s=.8))


In [ ]:
INPUT_DIR = "/home/floris/DB.Montreal/ArcTrace_Omega/201807_Simon"

In [ ]:
files = os.listdir(INPUT_DIR)
prefixes = []
suffixes = ["_parameters.json","_captured.pickle27","_trials.txt"]
for f in files:
    for s in suffixes:
        if f.endswith(s):
            prefixes.append(f[:-len(s)])
prefixes=list(set(prefixes))
prefixes.sort()

In [ ]:
# Let's take a look at how many trials are in each file ---
for p in prefixes:
    f = pd.read_csv(pj(INPUT_DIR,p+"_trials.txt"),sep=' ')
    print(p,f.shape[0])

Now from the options above, please select your input files, in the right order too.

In [ ]:
loadfiles = "Pierrick_arc_24_07.16h16m05 Pierrick_arc_24_07.16h22m36 Pierrick_arc_24_07.16h28m29 Pierrick_arc_24_07.16h35m53".split()

In [ ]:
#loadfiles = "Julie_arc_25_07.16h22m48 Julie_arc_25_07.16h29m38 Julie_arc_25_07.16h36m09 Julie_arc_25_07.16h43m09".split()

In [ ]:
#loadfiles = "marika_arc_26_07.14h51m55 marika_arc_26_07.14h58m00  marika_arc_26_07.15h04m23 marika_arc_26_07.15h10m00 ".split()

In [ ]:
#loadfiles = "Ibrahim_arc_27_07.15h11m15 Ibrahim_arc_27_07.15h17m09 Ibrahim_arc_27_07.15h22m51 Ibrahim_arc_27_07.15h30m05".split()

In [ ]:
alltrials = []
alltrajectories = {}
timestamps = []

cumul_trial = 0
for blocki,f in enumerate(loadfiles):
    tab = pd.read_csv(pj(INPUT_DIR,f+"_trials.txt"),sep=' ')
    tab["block"]=blocki+1 # the index of the block
    
    tab["cumul.trial"]=cumul_trial+tab["trial"]
    cumul_trial=max(tab["cumul.trial"])
    
    alltrials.append(tab)
    
    trial_to_cumul = dict(zip(tab["trial"],tab["cumul.trial"])) # which trial maps to which cumulative trial
    
    trajectories = pickle.load(open(pj(INPUT_DIR,f+"_captured.pickle27"),'rb'))
    
    # Now let's read the trajectories
    for traj in trajectories:
        tr = trial_to_cumul[traj["trial"]]
        thistraj = traj["trajectory"][1:] ### NOTE! Chopping off first data point because of glitch in the script

        # Convert trajectory to cm for ease of interpretation
        x,y,z=zip(*thistraj)
        thistraj = zip(100*array(x),100*array(y),100*array(z))
        
        alltrajectories[tr]=list(thistraj)
        timestamps.append({"cumul.trial":tr,"timestamp":traj["capture.t"]})
    
alltrials = pd.merge(pd.concat(alltrials),pd.DataFrame(timestamps))
alltrials["direction"]=['leftward' if tx<0 else 'rightward' for tx in alltrials["target.x"]]

# Also grab a parameter file while we are at it
params = json.load(open(pj(INPUT_DIR,f+'_parameters.json')))

In [ ]:
alltrials.head()

In [ ]:
blocks = list(set(alltrials["block"]))
print(blocks)

In [ ]:
# The prefix for figures
SUBJECT = alltrials.iloc[0]["participant"]
FIGDIR = 'figs/%s__'%SUBJECT
print(FIGDIR)

## How many trials were too slow (aborted)?

In [ ]:
for bl,data in alltrials.groupby('block'):
    n_incomplete = sum(data["timing"]=="incomplete.trial")
    perc = 100*n_incomplete/data.shape[0]
    print("Block %i   #incomplete = %i (%.2f %%)"%(bl,n_incomplete,perc))

In [ ]:
# Overall
n_incomplete = sum(alltrials["timing"]=="incomplete.trial")
perc = 100*n_incomplete/alltrials.shape[0]
print("Overall   #incomplete = %i (%.2f %%)"%(n_incomplete,perc))

### Remove the trajectories corresponding to these aborted trials


In [ ]:
for i,row in alltrials.iterrows():
    if row["timing"]=="incomplete.trial":
        tr = row["cumul.trial"]
        alltrajectories[tr]=[]

## Taking a look at individual trials

In [ ]:
def draw_arc(ax):
    """ 
    Draw a reference arc in a given plot
    """
    col   = 'black'
    alpha =.8
    lw    = 2

    circangs = linspace(0,2*pi,100)
    
    def arc(center,radius,angs):
        cx,cy=center
        ax.plot(100*(cx+radius*cos(angs)),
                100*(cy+radius*sin(angs)),color=col,alpha=alpha,lw=lw)
        # What that factor 100 is doing in there? Converting m to cm.

    arc(params["RIGHT_ARC_ORIGIN"],params["TARGET_RADIUS"],circangs)
    arc(params["LEFT_ARC_ORIGIN"] ,params["TARGET_RADIUS"],circangs)    
    
    arc((params["ARC_BASE_X"]-params["ARC_RADIUS_1"],params["ARC_BASE_Y"]),params["ARC_RADIUS_1"],linspace(0,   pi,100))
    arc((params["ARC_BASE_X"]+params["ARC_RADIUS_2"],params["ARC_BASE_Y"]),params["ARC_RADIUS_2"],linspace(pi,2*pi,100))

In [ ]:
f,ax = subplots(1,1,figsize=(10,5))
draw_arc(ax)
for i,row in alltrials.iterrows():  
    tr = row["cumul.trial"]
    traj = alltrajectories[tr]
    if len(traj):
        x,y,z=zip(*traj)
        plot(y[:],z[:],'-',lw=.2,color='blue')

ax.set_aspect('equal', 'datalim')
xlabel("y (cm)")
ylabel("z (cm)")
sns.despine(offset=5,trim=True)

### Chop it up by block

In [ ]:
blocks = list(set(alltrials["block"]))

f,axs = subplots(len(blocks),1,figsize=(10,15),sharex=True,sharey=True)
for i,bl in enumerate(blocks):
    axs[i].set_aspect('equal')
    axs[i].set_title("block %i"%bl)
    draw_arc(axs[i])

for i,row in alltrials.iterrows():  
    tr = row["cumul.trial"]
    bl = row["block"]
    ax = axs[blocks.index(bl)]
    traj = alltrajectories[tr]
    if len(traj):
        x,y,z=zip(*traj)
        ax.plot(y[:],z[:],'-',lw=.5,color='blue')
    
xlabel("y (cm)")
ylabel("z (cm)")
sns.despine(offset=5,trim=True)

### Chop it up by block and direction

In [ ]:
blocks = list(set(alltrials["block"]))
directions = list(set(alltrials["direction"]))

f,axs = subplots(len(blocks),len(directions),figsize=(15,15),sharex=True,sharey=True)
for i,bl in enumerate(blocks):
    for j,dr in enumerate(directions):
        axs[i][j].set_aspect('equal')
        axs[i][j].set_title("block %i %s"%(bl,dr))
        draw_arc(axs[i][j])

for i,row in alltrials.iterrows():  
    tr = row["cumul.trial"]
    bl = row["block"]
    dr = row["direction"]
    ax = axs[blocks.index(bl)][directions.index(dr)]
    traj = alltrajectories[tr]
    if len(traj):
        x,y,z=zip(*traj)
        ax.plot(y[:],z[:],'-',lw=.5,color='blue')

    
xlabel("y (cm)")
ylabel("z (cm)")
tight_layout()
sns.despine(offset=5,trim=True)
suptitle(SUBJECT)
savefig(FIGDIR+'raw_block_direction.pdf')

# Compute distances from arc

**TODO**

# Compute velocity, smoothness?

**TODO**